In [193]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import roc_auc_score

In [194]:
df = pd.read_csv('data/transactions.csv')
df.head()

,client_id,trans_time,mcc_code,trans_type,amount,term_id,trans_city
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,35 08:24:41,4829,2370,-1808.56,NaN,Tver
1,81c7349f7bdcb661bc7d5f5f0fdac6aa,351 13:27:15,6011,7010,7954.51,442207,Kazan
2,fc0c7c937f139bd6d1c5b5d0a0023dd3,150 16:03:59,6011,2010,-12293.56,NaN,Khabarovsk
3,3495487d5beaa5caaa0682fddc38379a,373 11:54:12,6010,7070,49.52,888987,Kaliningrad
4,07e279042b1da23710fa2f65034d30b9,340 13:11:45,6010,7070,45.29,888918,Saint Petersburg


In [195]:
genders = pd.read_csv('data/train.csv', encoding = "utf-8")

In [196]:
trans_types = pd.read_csv('data/trans_types.csv', encoding = "utf-8", sep=';')

In [197]:
genders1 = genders.drop('Unnamed: 0', axis = 1)

In [198]:
# df_w_genders = df.merge(genders1)
df_w_genders = df

In [199]:
df1 = df.drop(['term_id'], axis=1)
# df['term_id'] = df['term_id'].isna()
# df1
# df

In [200]:
# df1 = df.drop(['term_id'], axis=1)

In [201]:
def time_convertion_to_time(s):
    data = str(s).split()
    day = int(data[0])
    time = data[1]
    h, m, s = map(int, time.split(':'))
    return int(25200 < h * 3600 + m * 60 + s < 61200)


def time_convertion_to_h(s):
    data = str(s).split()
    day = int(data[0])
    time = data[1]
    h, m, s = map(int, time.split(':'))
    return h

def time_convertion_to_day(s):
    data = str(s).split()
    day = int(data[0])
    return day % 7

def day_binary(n):
    if n == 5 or n == 6: return 1
    return 0

In [202]:
df1['day'] = df1['trans_time'].apply(time_convertion_to_day)
df1['time'] = df1['trans_time'].apply(time_convertion_to_time)
df1['day_binary'] = df1['day'].apply(day_binary)
df1['h'] = df1['trans_time'].apply(time_convertion_to_h)

In [203]:
#
#
#
#

In [204]:
df2 = df1.drop(['trans_time'], axis=1)

In [205]:
df2.isna().sum()

client_id     0
mcc_code      0
trans_type    0
amount        0
trans_city    0
day           0
time          0
day_binary    0
h             0
dtype: int64

In [206]:
#
#
#
#
df2

,client_id,mcc_code,trans_type,amount,trans_city,day,time,day_binary,h
0,d1bbbc9a0e0410d3cf12a3d2f44f3450,4829,2370,-1808.56,Tver,0,1,0,8
1,81c7349f7bdcb661bc7d5f5f0fdac6aa,6011,7010,7954.51,Kazan,1,1,0,13
2,fc0c7c937f139bd6d1c5b5d0a0023dd3,6011,2010,-12293.56,Khabarovsk,3,1,0,16
3,3495487d5beaa5caaa0682fddc38379a,6010,7070,49.52,Kaliningrad,2,1,0,11
4,07e279042b1da23710fa2f65034d30b9,6010,7070,45.29,Saint Petersburg,4,1,0,13
...,...,...,...,...,...,...,...,...,...
3563524,e91858185ae43cf3970220748893a01b,6010,7070,227.40,Penza,6,1,1,13
3563525,429b6c8255f13bdb506fdbd6e342ccb1,4814,1030,-1084.44,Khabarovsk,3,0,0,19
3563526,9443b6a0d948cf368d8d72eb295bdfdf,5411,1110,-569.20,Tver,6,0,1,23
3563527,1be75d38ad89c0f2b4d85b95f0fd8069,5814,1110,-71.43,Vladivostok,3,0,0,23


In [207]:
df_enc = pd.get_dummies(df2, columns=['mcc_code', 'trans_type', 'trans_city', 'day'], dtype=int)
tmp = df_enc.groupby('client_id').sum()
stat = ['mean', 'count', 'min', 'max', 'median', 'sum'] # some features
df_AmtStatTmp = df_enc[['client_id', 'amount']].groupby('client_id').agg(stat).add_prefix('').droplevel(0, axis='columns')
df_AmtStatTmp=pd.merge(df_AmtStatTmp, tmp, right_index=True, left_index=True)

In [208]:
df_AmtStatTmp['client_id'] = df_AmtStatTmp.index

In [209]:
df_AmtStatTmp.index = np.arange(df_AmtStatTmp.shape[0])

In [210]:
#
#
#
#
#
#

In [211]:
df_AmtStatTmp1 = df_AmtStatTmp.merge(genders1, how='left')

In [212]:
df_AmtStatTmp1.isna().sum()

mean           0
count          0
min            0
max            0
median         0
            ... 
day_4          0
day_5          0
day_6          0
client_id      0
gender       840
Length: 288, dtype: int64

In [213]:
from sklearn.model_selection import train_test_split

X_final_test = df_AmtStatTmp1[df_AmtStatTmp1['gender'].isna()].drop(['client_id', 'gender'], axis=1)
X_train = df_AmtStatTmp1[~df_AmtStatTmp1['gender'].isna()]
y_train = X_train['gender']
X_train = X_train.drop(['gender', 'client_id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, train_size=0.8, random_state=0)


In [214]:
# X_all.isna().sum().sum()

In [215]:
X_all = pd.concat([X_train, X_test, X_final_test])
X_all

,mean,count,min,max,median,sum,amount,time,day_binary,h,...,trans_city_Tver,trans_city_Vladimir,trans_city_Vladivostok,day_0,day_1,day_2,day_3,day_4,day_5,day_6
434,200.172174,115,-16631.74,23863.15,-72.660,23019.80,23019.80,91,43,1527,...,0,0,0,21,11,6,15,19,25,18
7724,-445.020714,196,-10847.71,10846.18,-153.875,-87224.06,-87224.06,161,53,2443,...,0,0,0,30,28,26,23,36,32,21
6382,52.734510,51,-19523.32,25310.13,-108.100,2689.46,2689.46,31,20,674,...,51,0,0,13,4,2,6,6,12,8
1625,-620.132500,384,-21692.98,36155.74,-345.170,-238130.88,-238130.88,256,132,5637,...,0,0,0,63,44,46,53,46,64,68
5614,-193.727606,142,-3613.91,7232.08,-140.180,-27509.32,-27509.32,72,39,1357,...,0,0,0,17,21,26,21,18,22,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8339,-1223.119097,144,-39771.24,36156.98,-616.095,-176129.15,-176129.15,93,42,1999,...,0,0,0,20,38,18,14,12,25,17
8356,-125.419566,346,-2819.92,1446.05,-71.805,-43395.17,-43395.17,194,111,4463,...,0,0,0,55,29,33,66,52,46,65
8369,-382.577432,370,-18078.23,36156.18,-140.775,-141553.65,-141553.65,204,104,5356,...,370,0,0,56,56,56,47,51,58,46
8377,-1232.901375,269,-32540.15,14462.57,-212.940,-331650.47,-331650.47,144,88,3778,...,0,0,269,50,36,25,26,44,37,51


In [216]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(X_all)
# X_train_normed = scaler.transform(X_train)
# X_test_normed = scaler.transform(X_test)
# X_final_test_normed = scaler.transform(X_final_test)

In [217]:
from sklearn import preprocessing


scaler = preprocessing.MinMaxScaler(feature_range=(-2, 2))
scaler.fit(X_all)
X_train_normed = scaler.transform(X_train)
X_test_normed = scaler.transform(X_test)
X_final_test_normed = scaler.transform(X_final_test)

In [218]:
X_train_normed

array([[ 0.28418058, -1.99459152,  1.978037  , ..., -1.99437204,
        -1.9930396 , -1.99497803],
       [ 0.26282828, -1.99074865,  1.98336058, ..., -1.98933649,
        -1.99109069, -1.99414103],
       [ 0.27930121, -1.99762786,  1.9753756 , ..., -1.99822275,
        -1.99665901, -1.99776801],
       ...,
       [ 0.17884895, -1.97561438,  1.95094839, ..., -1.97748815,
        -1.97856198, -1.9818651 ],
       [ 0.09243755, -1.99857672,  1.98003288, ..., -1.99822275,
        -1.99832951, -1.999442  ],
       [ 0.2717298 , -1.99487617,  1.98981793, ..., -1.99496445,
        -1.99693743, -1.99609402]])

In [219]:
#ааааааааааааааа нахуй ML наконец то

In [220]:
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier

gbc1 = HistGradientBoostingClassifier(random_state=42, max_iter = 500, learning_rate=0.05, max_depth=3)

gbc1.fit(X_train_normed, y_train)
preds_valid_gbc1 = gbc1.predict_proba(X_test_normed)
roc_auc_score(y_test, [i[1] for i in preds_valid_gbc1])

0.8750097493529974

In [221]:
# from sklearn.ensemble import GradientBoostingClassifier


# gbc = GradientBoostingClassifier(random_state=240, n_estimators=600, learning_rate=0.05, loss = 'exponential', max_depth=3)

# gbc.fit(X_train_normed, y_train)
# preds_valid_gbc = gbc.predict_proba(X_test_normed)
# roc_auc_score(y_test, [i[1] for i in preds_valid_gbc])

In [222]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=240, verbosity=0, eta = 0.14, max_depth = 3, n_estimators=285)
xgb.fit(X_train_normed, y_train)
preds_valid_xgb = xgb.predict_proba(X_test_normed)
# preds_valid_xgb = np.array([[1 - i, i] for i in preds_valid_xgb])
roc_auc_score(y_test, [i[1] for i in preds_valid_xgb])

0.8730545609245932

In [223]:
from xgboost import XGBClassifier

xgb1 = XGBClassifier(eval_metric='auc', random_state=42, verbosity=0, eta = 0.14, max_depth = 3, n_estimators=500, learning_rate = 0.05)
xgb1.fit(X_train_normed, y_train)
preds_valid_xgb1 = xgb1.predict_proba(X_test_normed)
# preds_valid_xgb = np.array([[1 - i, i] for i in preds_valid_xgb])
roc_auc_score(y_test, [i[1] for i in preds_valid_xgb1])

0.8759979792250151

In [224]:
df_enc = pd.get_dummies(df2, columns=['mcc_code', 'trans_type', 'trans_city', 'day', 'h'], dtype=int)
tmp = df_enc.groupby('client_id').sum()
stat = ['mean', 'count', 'min', 'max', 'median', 'sum'] # some features
df_AmtStatTmp = df_enc[['client_id', 'amount']].groupby('client_id').agg(stat).add_prefix('').droplevel(0, axis='columns')
df_AmtStatTmp=pd.merge(df_AmtStatTmp, tmp, right_index=True, left_index=True)

In [225]:
df_AmtStatTmp['client_id'] = df_AmtStatTmp.index

In [226]:
df_AmtStatTmp.index = np.arange(df_AmtStatTmp.shape[0])

In [227]:
df_AmtStatTmp1 = df_AmtStatTmp.merge(genders1, how='left')

In [228]:
from sklearn.model_selection import train_test_split

X_final_test1 = df_AmtStatTmp1[df_AmtStatTmp1['gender'].isna()].drop(['client_id', 'gender'], axis=1)
X_train1 = df_AmtStatTmp1[~df_AmtStatTmp1['gender'].isna()]
y_train1 = X_train1['gender']
X_train1 = X_train1.drop(['gender', 'client_id'], axis=1)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train1, y_train1, train_size=0.8, random_state=0)
X_all1 = pd.concat([X_train1, X_test1, X_final_test1])
X_all1

,mean,count,min,max,median,sum,amount,time,day_binary,mcc_code_742,...,h_14,h_15,h_16,h_17,h_18,h_19,h_20,h_21,h_22,h_23
434,200.172174,115,-16631.74,23863.15,-72.660,23019.80,23019.80,91,43,0,...,12,10,13,7,3,5,6,0,1,1
7724,-445.020714,196,-10847.71,10846.18,-153.875,-87224.06,-87224.06,161,53,0,...,14,16,19,15,16,1,0,0,0,0
6382,52.734510,51,-19523.32,25310.13,-108.100,2689.46,2689.46,31,20,0,...,1,4,1,0,1,0,3,3,1,6
1625,-620.132500,384,-21692.98,36155.74,-345.170,-238130.88,-238130.88,256,132,0,...,38,30,22,27,18,21,22,17,13,6
5614,-193.727606,142,-3613.91,7232.08,-140.180,-27509.32,-27509.32,72,39,0,...,9,6,2,0,0,0,0,0,0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8339,-1223.119097,144,-39771.24,36156.98,-616.095,-176129.15,-176129.15,93,42,0,...,10,13,6,5,7,8,11,8,3,2
8356,-125.419566,346,-2819.92,1446.05,-71.805,-43395.17,-43395.17,194,111,0,...,17,25,38,33,21,3,3,0,0,40
8369,-382.577432,370,-18078.23,36156.18,-140.775,-141553.65,-141553.65,204,104,0,...,17,32,33,27,40,47,19,5,2,7
8377,-1232.901375,269,-32540.15,14462.57,-212.940,-331650.47,-331650.47,144,88,0,...,18,13,23,23,27,14,9,5,0,20


In [229]:
X_final_test1

,mean,count,min,max,median,sum,amount,time,day_binary,mcc_code_742,...,h_14,h_15,h_16,h_17,h_18,h_19,h_20,h_21,h_22,h_23
5,-620.716613,124,-8247.26,10847.44,-473.660,-76968.86,-76968.86,114,27,0,...,11,6,5,3,2,0,0,0,0,0
9,-638.637603,267,-19885.46,3978.15,-72.570,-170516.24,-170516.24,173,58,0,...,5,15,66,34,14,5,3,5,4,11
13,-276.066986,355,-36156.04,72312.69,-72.140,-98003.78,-98003.78,262,119,0,...,27,23,7,8,9,5,7,6,17,7
40,-592.139910,553,-22417.67,7592.34,-208.790,-327453.37,-327453.37,365,161,0,...,37,34,36,29,26,14,14,5,3,38
45,-170.163333,300,-15546.56,21692.85,-110.685,-51049.00,-51049.00,206,57,0,...,3,2,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8339,-1223.119097,144,-39771.24,36156.98,-616.095,-176129.15,-176129.15,93,42,0,...,10,13,6,5,7,8,11,8,3,2
8356,-125.419566,346,-2819.92,1446.05,-71.805,-43395.17,-43395.17,194,111,0,...,17,25,38,33,21,3,3,0,0,40
8369,-382.577432,370,-18078.23,36156.18,-140.775,-141553.65,-141553.65,204,104,0,...,17,32,33,27,40,47,19,5,2,7
8377,-1232.901375,269,-32540.15,14462.57,-212.940,-331650.47,-331650.47,144,88,0,...,18,13,23,23,27,14,9,5,0,20


In [230]:
from sklearn import preprocessing


scaler1 = preprocessing.MinMaxScaler(feature_range=(-3, 3))
scaler1.fit(X_all1)
X_train_normed1 = scaler1.transform(X_train1)
X_test_normed1 = scaler1.transform(X_test1)
X_final_test_normed1 = scaler1.transform(X_final_test1)

In [231]:
from xgboost import XGBClassifier


di={'max_depth': 7, 'learning_rate': 0.002866308072110825, 'min_child_weight': 42, 'gamma': 0.04755879097801036, 'subsample': 0.7073502864471638,
        'colsample_bynode': 0.7855717372496582, 'colsample_bytree': 0.8097240402391387, 'reg_alpha': 0.0010326613711012255, 'reg_lambda': 4.315201937021431e-07}


xgb2 = XGBClassifier(eval_metric='auc', n_estimators=5300, random_state=42, **di)
xgb2.fit(X_train_normed1, y_train1)
preds_valid_xgb2 = xgb2.predict_proba(X_test_normed1)
# preds_valid_xgb = np.array([[1 - i, i] for i in preds_valid_xgb])
roc_auc_score(y_test1, [i[1] for i in preds_valid_xgb2])

0.879976601552806

In [232]:
# from xgboost import XGBClassifier


# di={'max_depth': 7, 'learning_rate': 0.002866308072110825, 'min_child_weight': 42, 'gamma': 0.04755879097801036, 'subsample': 0.7073502864471638,
#         'colsample_bynode': 0.7855717372496582, 'colsample_bytree': 0.8097240402391387, 'reg_alpha': 0.0010326613711012255, 'reg_lambda': 5.315201937021431e-07}


# xgb = XGBClassifier(eval_metric='auc', n_estimators=5000, random_state=42, **di)
# xgb.fit(X_train_normed, y_train)
# preds_valid_xgb = xgb.predict_proba(X_test_normed)
# # preds_valid_xgb = np.array([[1 - i, i] for i in preds_valid_xgb])
# roc_auc_score(y_test, [i[1] for i in preds_valid_xgb])

In [233]:
from sklearn.preprocessing import StandardScaler

X_train_ml = X_train
y_train_ml = y_train
X_test_ml = X_test
y_test_ml = y_test
X_final_test_ml = X_final_test

scaler_ml = StandardScaler()
scaler_ml.fit(X_all)
X_train_normed_ml = scaler_ml.transform(X_train_ml)
X_test_normed_ml = scaler_ml.transform(X_test_ml)
X_final_test_normed_ml = scaler_ml.transform(X_final_test_ml)

In [234]:
import torch
from torch import nn
import torch.nn.functional as F

# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [235]:
class MyModel1(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=286, out_features=50)
        self.dropout = nn.Dropout(p=0.1)
        self.layer_2 = nn.Linear(in_features=50, out_features=25)
        self.layer_3 = nn.Linear(in_features=25, out_features=12)
        self.layer_4 = nn.Linear(in_features=12, out_features=1)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.dropout(x)
        x = self.layer_2(F.relu(x))
        x = self.dropout(x)
        x = self.layer_3(F.relu(x))
        x = self.dropout(x)
        x = self.layer_4(F.tanh(x))
        return x
    
# 4. Create an instance of the model and send it to target device
model_0 = MyModel1().to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.ASGD(params=model_0.parameters(), 
                            lr=0.1)

def accuracy_fn(y_true, y_pred):
    return roc_auc_score(y_true.detach().numpy(), y_pred.detach().numpy())


X_train_ml = torch.tensor(X_train_normed_ml).to(torch.float32)
X_test_ml = torch.tensor(X_test_normed_ml).to(torch.float32)
y_train_ml = torch.tensor(np.array(y_train_ml)).to(torch.float32)
y_test_ml = torch.tensor(np.array(y_test_ml)).to(torch.float32)

In [236]:
torch.manual_seed(42)

epochs = 600

X_train_ml, y_train_ml = X_train_ml.to(device), y_train_ml.to(device)
X_test_ml, y_test_ml = X_test_ml.to(device), y_test_ml.to(device)

for epoch in range(epochs):
    model_0.train()

    y_logits = model_0(X_train_ml).squeeze()
    
    y_pred = y_logits
    
    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train_ml) 
    
    acc = accuracy_fn(y_true=y_train_ml, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_0(X_test_ml).squeeze() 
        test_pred = test_logits
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test_ml)
        test_acc = accuracy_fn(y_true=y_test_ml,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Test acc: {test_acc}")

Epoch: 0 | Test acc: 0.5392260786329635
Epoch: 10 | Test acc: 0.567309533094622
Epoch: 20 | Test acc: 0.595135959159074
Epoch: 30 | Test acc: 0.622867550608005
Epoch: 40 | Test acc: 0.645663310525756
Epoch: 50 | Test acc: 0.6645212181373418
Epoch: 60 | Test acc: 0.6769348034175913
Epoch: 70 | Test acc: 0.6858386216187471
Epoch: 80 | Test acc: 0.6931789981210337
Epoch: 90 | Test acc: 0.7003297053922786
Epoch: 100 | Test acc: 0.7089569964902329
Epoch: 110 | Test acc: 0.71760910412309
Epoch: 120 | Test acc: 0.7257914702024321
Epoch: 130 | Test acc: 0.7341209628815543
Epoch: 140 | Test acc: 0.7430939128585103
Epoch: 150 | Test acc: 0.7535877619030736
Epoch: 160 | Test acc: 0.7647800191441841
Epoch: 170 | Test acc: 0.7776225759563229
Epoch: 180 | Test acc: 0.7893714326231076
Epoch: 190 | Test acc: 0.8012266458680469
Epoch: 200 | Test acc: 0.8106179317190769
Epoch: 210 | Test acc: 0.8178164994504911
Epoch: 220 | Test acc: 0.8243432481298969
Epoch: 230 | Test acc: 0.8294572269294856
Epoch: 24

In [237]:
lst = []
for a in range(2):
    for b in range(10, 30):
        for c in range(8):
            for e in range(3):
                lst.append([roc_auc_score(y_test, (a * np.array([i[1] for i in preds_valid_xgb])
     + b * np.array([i[1] for i in preds_valid_xgb2])
     + c * np.array([i[1] for i in preds_valid_gbc1])
     + e * model_0(X_test_ml).squeeze().detach().numpy()   )/(a + b + c + e)), a, b, c, e])
print(max(lst))

[0.8874162441947036, 0, 29, 4, 2]


In [238]:
# lst = []
# for a in range(15):
#     for b in range(1, 25):
#         for d in range(5):
#             lst.append([roc_auc_score(y_test, (a * np.array([i[1] for i in preds_valid_xgb1])
#      + b * np.array([i[1] for i in preds_valid_xgb])
#      + c * np.array([i[1] for i in preds_valid_gbc])
#      + d * model_0(X_test).squeeze().detach().numpy())/(a + b + d)), a, b, d])
# print(max(lst))

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame

In [ ]:
pd.DataFrame(ans).to_csv("result .csv")